# 方案：引入时间损失以监督识别周期

## 1. 目标

将模型的任务一分为二：
1.  **主任务（分类）**：正确识别目标类别（已存在）。
2.  **辅助任务（时间预测）**：预测出识别该目标所需的“理想帧数”。

通过增加这个辅助任务，模型在努力提升分类准确率的同时，也被激励去学习如何更高效地利用信息，从而在推理时能够更快地达到决策阈值。

## 2. 步骤

核心步骤：修改模型、修改数据加载器、修改训练循环。

### 第1步：修改模型结构 (`FusedModel`)

我们需要将原有的单输出模型，修改为一个拥有两个独立“头”的双输出模型。

In [ ]:
import torch
import torch.nn as nn

# 假设的 rd_model 和 track_model，用于演示
class DummyRDModel(nn.Module):
    def __init__(self): 
        super().__init__()
        self.num_features = 128
    def forward(self, images, image_mask=None): return torch.randn(images.shape[0], self.num_features)

class DummyTrackModel(nn.Module):
    def __init__(self): 
        super().__init__()
        self.d_model = 256
    def forward(self, features, track_mask=None): return torch.randn(features.shape[0], self.d_model)

class FusedModelMultiTask(nn.Module):
    """支持多任务学习的融合模型"""
    def __init__(self, rd_model, track_model, num_classes):
        super().__init__()
        self.swin_transformer = rd_model
        self.roformer = track_model
        
        # 融合后的特征维度
        fused_dim = self.swin_transformer.num_features + self.roformer.d_model
        
        # 任务1: 分类头
        self.classification_head = nn.Linear(fused_dim, num_classes)
        
        # 任务2: 时间预测头 (输出一个标量)
        self.time_prediction_head = nn.Sequential(
            nn.Linear(fused_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, track_features, images, track_mask=None, image_mask=None):
        track_features = self.roformer(track_features, track_mask)
        image_features = self.swin_transformer(images, image_mask)
        
        features = torch.cat([track_features, image_features], dim=1)
        
        # 得到两个独立的输出
        classification_output = self.classification_head(features)
        time_prediction_output = self.time_prediction_head(features).squeeze(-1) # squeeze() 移除多余维度
        
        return classification_output, time_prediction_output

### 第2步：修改数据加载 (`FusedDataset`)

数据集的 `__getitem__` 方法需要额外返回“理想帧数”这个学习目标。现有阶段这个值固定为10，即每一类目标的“理想分类所花的周期数”。

In [ ]:
from torch.utils.data import Dataset

class DummyDataset(Dataset):
    """模拟的数据集，用于演示返回额外的时间标签"""
    def __init__(self, num_samples=100, num_classes=6):
        self.num_samples = num_samples
        self.labels = torch.randint(0, num_classes, (num_samples,))

    def __len__(self):
        return self.num_samples

    def __getitem__(self, item):
        # 模拟返回的数据
        images = torch.randn(64, 1, 32, 544) # 固定长度以简化演示
        tracks = torch.randn(20, 15)
        classification_label = self.labels[item]
        
        # **核心改动**: 返回固定的理想帧数标签
        ideal_timestep_label = torch.tensor(10.0)
        
        return images, tracks, classification_label, ideal_timestep_label

### 第3步：修改训练循环

训练逻辑需要更新，以计算并组合两个任务的损失。

In [ ]:
from torch.utils.data import DataLoader

# --- 1. 初始化 --- 
batch_size = 16
num_classes = 6
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 实例化模型和数据加载器
model = FusedModelMultiTask(DummyRDModel(), DummyTrackModel(), num_classes).to(device)
dataset = DummyDataset(num_samples=batch_size*4)
loader = DataLoader(dataset, batch_size=batch_size)

# 定义两个损失函数
criterion_cls = nn.CrossEntropyLoss() # 分类损失
criterion_time = nn.MSELoss()         # 时间预测损失 (均方误差)

# 定义优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 定义时间损失的权重 (这是一个可以调整的超参数)
time_loss_weight = 0.1

# --- 2. 模拟训练循环 --- 
model.train()
for images, tracks, cls_labels, time_labels in loader:
    # 将数据移动到设备
    images, tracks = images.to(device), tracks.to(device)
    cls_labels, time_labels = cls_labels.to(device), time_labels.to(device)
    
    # 1. 前向传播，获取两个输出
    cls_output, time_output = model(tracks, images)
    
    # 2. 分别计算两个损失
    loss_classification = criterion_cls(cls_output, cls_labels)
    loss_time = criterion_time(time_output, time_labels)
    
    # 3. 计算加权总损失
    total_loss = loss_classification + time_loss_weight * loss_time
    
    # 4. 反向传播和优化
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    
    print(f"批次完成 | 总损失: {total_loss.item():.4f} | 分类损失: {loss_classification.item():.4f} | 时间损失: {loss_time.item():.4f}")

print("\n模拟训练完成。")

## 3. 总结与后续

通过上述三个步骤，我们成功地构建了一个多任务学习框架。模型在学习如何正确分类的同时，也被一个额外的“时间损失”监督着，激励它去学习在10帧左右就能做出判断的内部特征。

**后续工作**：
1.  将此逻辑应用于真实的项目代码中。
2.  在 `fusion/configs/fused.yaml` 中添加 `time_loss_weight` 超参数，方便调整。
3.  在推理阶段，可以利用模型的 `time_prediction_output` 输出来辅助制定“早退”策略，例如，当模型的分类置信度足够高，且其预测的所需帧数接近或小于当前已用帧数时，即可做出最终决策。